In [1]:
import os

In [4]:
folder_name = "known_faces"

In [5]:
if not os.path.exists(folder_name):
    os.makedirs(folder_name)  # Create the folder
    print(f"Created directory: {folder_name}")
else:
    print(f"Directory already exists: {folder_name}")


Created directory: known_faces


In [2]:
import cv2

cam = cv2.VideoCapture(0)
ret, frame = cam.read()
if ret:
    cv2.imwrite("known_faces/bhuvanesh.jpg", frame)
cam.release()
cv2.destroyAllWindows()
print("✅ Saved new face to known_faces")


✅ Saved new face to known_faces


In [1]:
import os

print(os.getcwd())  # shows your working directory
print("📁 known_faces content:", os.listdir('known_faces'))


C:\Windows\System32
📁 known_faces content: ['bhuvanesh.jpg']


In [7]:
import os

# Set the working directory to your actual project path
os.chdir("C:/Users/abhuv/Downloads/face_attendence_project")

# Confirm it's correct
print("✅ Current directory:", os.getcwd())
print("📁 known_faces content:", os.listdir("known_faces"))


✅ Current directory: C:\Users\abhuv\Downloads\face_attendence_project
📁 known_faces content: ['bhuvanesh.jpg', 'naveen.jpg']


In [8]:
import face_recognition
import os

known_faces_dir = "known_faces"

for filename in os.listdir(known_faces_dir):
    if filename.endswith(('.jpg', '.png')):
        path = os.path.join(known_faces_dir, filename)
        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            print(f"✅ Face detected in {filename}")
        else:
            print(f"❌ No face found in {filename} — try retaking or repositioning the image.")


✅ Face detected in bhuvanesh.jpg
✅ Face detected in naveen.jpg


In [11]:
import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime
import pandas as pd

# Make sure you're already in the correct directory
known_faces_dir = "known_faces"

known_encodings = []
known_names = []

print("🔄 Loading known faces...")

for filename in os.listdir(known_faces_dir):
    if filename.endswith(('.jpg', '.png')):
        path = os.path.join(known_faces_dir, filename)
        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            known_encodings.append(encodings[0])
            known_names.append(os.path.splitext(filename)[0])
        else:
            print(f"⚠️ No face found in {filename}")

print("✅ Known faces loaded!")

cap = cv2.VideoCapture(0)
students = known_names.copy()
attendance = []

print("📷 Starting camera... Press 'q' to stop and save attendance.")

while True:
    success, frame = cap.read()
    if not success:
        break

    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)

        name = "Unknown"
        if len(face_distances) > 0:
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_names[best_match_index]
                if name in students:
                    students.remove(name)
                    now = datetime.now()
                    time_string = now.strftime('%H:%M:%S')
                    attendance.append([name, time_string])
                    print(f"✅ Marked {name} at {time_string}")

        # Draw rectangle and name
        top, right, bottom, left = [v * 4 for v in face_location]
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Show live frame
    cv2.imshow("Face Attendance", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save attendance
df = pd.DataFrame(attendance, columns=["Name", "Time"])
df.to_csv("attendance.csv", index=False)
print("📁 Attendance saved to attendance.csv")


🔄 Loading known faces...
✅ Known faces loaded!
📷 Starting camera... Press 'q' to stop and save attendance.
✅ Marked bhuvanesh at 19:43:40
📁 Attendance saved to attendance.csv
